# 7. Bokeh_Widgets

In [ ]:
# 加载
import pandas as pd
import numpy as np
# 导入notebook绘图模块

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import Panel, Tabs, CustomJS, ColumnDataSource,WidgetBox,HoverTool,Button, RadioButtonGroup, Select, Slider,CheckboxGroup, CustomJSFilter, CDSView, IndexFilter
from bokeh.core.properties import value
from bokeh.models.annotations import Span
from bokeh.layouts import column,gridplot,row
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

In [ ]:
## Import data source'
dict_source={'LB':'lb.csv'}
for k,v in dict_source.items():
    globals()['df_'+k]=pd.read_csv('./data/bokeh/'+v)  

In [ ]:
def UnstackToFatDf(dfn,lst_key,lst_var,lv):
    df_temp=dfn.copy()
    df_temp=df_temp[lst_key+lst_var]
    df_temp=df_temp.set_index(lst_key)
    df_temp_unstack=df_temp.unstack(level=lv)
    df_temp_unstack.columns=df_temp_unstack.columns.droplevel()
    df_temp_unstack=df_temp_unstack.reset_index()
    return df_temp_unstack

In [ ]:
df_eDISH=df_LB[df_LB['AnalyteName'].isin(['AST','ALT','BILI'])]
df_eDISH['VALUE/ULN']=df_eDISH['NumericValue']/df_eDISH['LabHigh']
df_eDISH_Peak=df_eDISH.groupby(['Subject','AnalyteName'])['VALUE/ULN'].max().reset_index()
df_eDISH_Peak_Unstack=UnstackToFatDf(df_eDISH_Peak,['Subject','AnalyteName'],['VALUE/ULN'],1)
df_eDISH_Peak_Unstack=df_eDISH_Peak_Unstack.rename(columns={'AST':'Peak_AST','ALT':'Peak_ALT','BILI':'Peak_BILI'})
df_eDISH_Peak_Unstack=df_eDISH_Peak_Unstack.reset_index()
# df_eDISH_Peak_Unstack.fillna(0,inplace=True)
# df_eDISH_Peak_Unstack
lst_Subject=df_eDISH_Peak_Unstack['Subject'].tolist()
# lst_Subject

In [ ]:
def modify_doc(doc):
    
    def make_dataset(lst_subset):       
        
        lst_cols=list(df_eDISH_Peak_Unstack.columns)
        df_eDISH_Peak_Unstack_subset=pd.DataFrame(columns=lst_cols)    
        
        for i,pt in enumerate(lst_subset):
            df_subset=df_eDISH_Peak_Unstack[df_eDISH_Peak_Unstack['Subject']==pt]
            
            df_eDISH_Peak_Unstack_subset=df_eDISH_Peak_Unstack_subset.append(df_subset)
        
        return ColumnDataSource(df_eDISH_Peak_Unstack_subset)
    
    def style(p1):
        p1.background_fill_color = "beige"    # 绘图空间背景颜色
        p1.background_fill_alpha = 0.5        # 绘图空间背景透明度

        p1.border_fill_color = "whitesmoke"    # 外边界背景颜色
        p1.border_fill_alpha = 0.5             #透明度
        p1.min_border_left = 80                # 外边界背景 - 左边宽度
        p1.min_border_right = 80               # 外边界背景 - 右边宽度
        p1.min_border_top = 10                 # 外边界背景 - 上宽度
        p1.min_border_bottom = 10 

        p1.xaxis.axis_label = "Peak_AST"
        p1.xaxis.axis_line_width = 0.5
        # p1.xaxis.bounds = (0, 3)

        p1.yaxis.axis_label = "Peak_BILI"
        # p1.yaxis.major_label_text_color = "orange"
        # p1.yaxis.major_label_orientation = "vertical"

        # p1.axis.minor_tick_in = 20     # 刻度往绘图区域内延伸长度；设置成负的就是往外边延伸了。
        # p1.axis.minor_tick_out = 3   # 刻度往绘图区域外延伸长度

        upper1 = Span(location=3,           # 设置位置，对应坐标值
                     dimension='width',    # 设置方向，width为横向，height为纵向  
                     line_color='olive', line_width=2,   # 设置线颜色、线宽
                     line_dash = [8,4]
                    )
        upper2 = Span(location=2,dimension='height',line_color='firebrick', line_width=2,line_dash = [8,4])
        p1.add_layout(upper1)
        p1.add_layout(upper2)       

        return p1
    
    def make_plot(src):
        hover = HoverTool(tooltips=[("Peak_AST", "@Peak_AST"),
                                    ("Peak_BILI", "@Peak_BILI"),
                                    ("Subject","@Subject")
                                    ])
        
        p1 = figure(plot_width=800, plot_height=300,toolbar_location="above",
                    x_range=(0,5),
                    y_range=(0,6),
                    tools=[hover,'box_select,reset,wheel_zoom,pan,crosshair'],
                    title="eDISH"
                   )   # 注意这里书写方式;  hover它的作用是只是会显示出点的每个标签;crossshair是显示十字叉
        
        p1.circle(x = 'Peak_AST',
                  y = 'Peak_BILI',
                  source = src,
                  size = 7,
        #           alpha = 0.3        
                 )     
        
              
        
        # Styling
        p1 = style(p1)

        return p1
    
    def update(attr, old, new):
        if subject_selection1.value in lst_Subject:
            subjects_to_plot=[subject_selection1.value]        
        else:
            subjects_to_plot=lst_Subject
        
        # subjects_to_plot = [subject_selection.labels[i] for i in subject_selection.active]        
        
        new_src = make_dataset(subjects_to_plot,
                              )

        src.data.update(new_src.data)
        
    # subject_selection = CheckboxGroup(labels=lst_Subject, active = [0, 1])
    # subject_selection.on_change('active', update)
    
    ######
    subject_selection1 = Select(options=lst_Subject+['All'])
    subject_selection1.on_change('value', update)     
    
    
    
    # initial_subjects = [subject_selection.labels[i] for i in subject_selection.active]
    initial_subjects = [subject_selection1.options[3]]
    
    src = make_dataset(initial_subjects,
                      )
    
    p1 = make_plot(src)
    
    # Put controls in a single element
    controls = column(subject_selection1,
                        )
    
    # Create a row layout
    layout = column(controls, p1)
    
    # Make a tab with the layout 
    tab = Panel(child=layout, title = 'eDISH')
    tabs = Tabs(tabs=[tab])
    
    doc.add_root(tabs)
    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [ ]:
show(app, )